In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains  import ActionChains
from time import sleep

from PIL import Image
import pytesseract
import numpy as np
# path to your tesseract.exe
pytesseract.pytesseract.tesseract_cmd = '{path to tesseract}/tesseract.exe'

from playsound import playsound

# Start Browser

In [2]:
option = webdriver.ChromeOptions()
# path to my browser
option.binary_location = '{path to browser exe}'
driver = webdriver.Chrome(options=option)

# or you can call chrome by uncommenting the code below and comment the code above
# driver = webdriver.Chrome()

# Login

## Login actions

In [3]:
def login_and_play(driver):
    driver.get('https://pokemon-planet.com/')

    # click playnow button
    driver.find_element(By.ID, 'playnow').click()
    # input username and password
    driver.find_element(By.ID, 'user').send_keys('{username}')
    driver.find_element(By.ID, 'passwrd').send_keys('{password}')
    # click login
    driver.find_element(By.CLASS_NAME, 'loginbutton').click()
    # play
    driver.find_element(By.ID, 'playnowbutton').click()
    # wait for the game to load
    sleep(50)

    driver.find_element(By.ID, 'unity-fullscreen-button').click()


# Reload game

In [4]:
# reload the game window incase of connection error
# returns a pointer to game canvas
def reload(driver):
    driver.get('https://pokemon-planet.com/forums/game/')
    sleep(30)
    driver.find_element(By.ID, 'unity-fullscreen-button').click()
    return driver.find_element(By.ID, 'unity-canvas')

# Game Action

In [5]:
def hop_on_bike(driver, game):
    hop_on_bike = ActionChains(driver)
    hop_on_bike.move_to_element(game)
    hop_on_bike.send_keys('B')
    # perform the action
    hop_on_bike.perform()

In [6]:
# walking function that walks left and right
def walking(driver, game) :
    walk = ActionChains(driver)
    walk.move_to_element(game)
    walk.click_and_hold(game)

    # Simulate press-and-hold of the "B" key for 2 seconds
    walk.key_down('a').pause(1)
    walk.key_up('a')
    walk.key_down('d').pause(1)
    walk.key_up('d')
    walk.perform()

# Encounters A Pokemon

## Screenshot

In [7]:
def screen_capture(driver):
    driver.get_screenshot_as_file('screenshot.png')

## Screenshot battle log

Edit crop pixel location according to screen size

In [8]:
def battle_log(driver):
    screen_capture(driver)
    image = Image.open('screenshot.png')

    # crop by pixel location
    cropped_image = image.crop((300, 220, 
                                700, 750))

    cropped_image.save("screenshot.png")

## Recognize text

In [9]:
def text_recog():
    filename = 'screenshot.png'
    img1 = np.array(Image.open(filename))
    text = pytesseract.image_to_string(img1)
    return text

## Screenshot and recognize text within image

This is used to capture game error keywords

In [10]:
def text_screenshot(driver):
    screen_capture(driver)
    return text_recog()

# Battle Actions

## Run

In [11]:
def run(driver, game):
    run_action = ActionChains(driver)
    run_action.move_to_element(game)
    run_action.send_keys('4')
    # perform the action
    run_action.perform()
    sleep(1)
    run_action.perform()

## Fight

Uses the first move of the pokemon

In [12]:
def fight(driver, game):
    fight_action = ActionChains(driver)
    fight_action.move_to_element(game)
    fight_action.send_keys('1').pause(0.5)
    fight_action.send_keys('1')
    # perform the action
    fight_action.perform()

## Catch

Notifies the user to catch the desired pokemon

In [13]:
def catch():
    # for playing note.wav file
    try:
        playsound(r'./bubble-pop.wav')
    except:
        catch()

## Potion

Potion location relative to backpack location. Edit this function accordingly.

In [14]:
def use_potion(driver, game):
    potion_action = ActionChains(driver)
    potion_action.move_to_element(game)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('2').pause(1)
    potion_action.send_keys('3')
    # perform the action
    potion_action.perform()
    sleep(0.5)

## Recover Move

Move relative to the pokemon in the battle... Edit accordingly

In [15]:
def use_recover_move(driver, game):
    fight_action = ActionChains(driver)
    fight_action.move_to_element(game)
    fight_action.send_keys('1').pause(0.5)
    fight_action.send_keys('4').pause(7)
    # perform the action
    fight_action.perform()

# Automation

## Play action

In [16]:
def play(driver, run_poke, fight_poke, catch_poke, reload_words):
    encounter = 0
    global game
    while True:
        walking(driver, game)
        walking(driver, game)
        walking(driver, game)
        battle_log(driver)
        text = text_recog()
        for pokemon in catch_poke:
            if pokemon in text:
                catch()
                break
        for pokemon in run_poke:
            if pokemon in text:
                # potion/recovery move use to account for failed runs
                # uncomment as needed
                # if encounter > 10:  # adjust encounter count according to need
                #     usePotion(driver, game)
                #     # useRecoverMove(driver, game)
                #     encounter = 0
                run(driver, game)
                encounter += 1
                break
        for pokemon in fight_poke:
            if pokemon in text:
                # useRecoverMove(driver)
                fight(driver, game)
                break
        for keyword in reload_words:
            if keyword in text_screenshot(driver):
                game = reload(driver)
                hop_on_bike(driver, game)
                print('game disconnected, reloading')
        

# Execution

In [17]:
# define a list of pokemons that may appear
run_poke = []  # run away from those pokemons
fight_poke = ['Elite']  # fight those pokemons
catch_poke = ['Shiny']

reload_words = ['Disconnected', 'Login failed', 'Invalid', 'Unable to connect']

In [18]:
login_and_play(driver)
game = driver.find_element(By.ID, 'unity-canvas')
play(driver, run_poke, fight_poke, catch_poke, reload_words)

In [ ]:
driver.quit()
print("end...")

end...
